In [ ]:
#发送请求

In [4]:
APIURL = "https://c1544mq3nj.execute-api.us-east-1.amazonaws.com/prod"

In [6]:
!curl -X POST -H 'Content-Type: application/json' $APIURL --data '@./payload1.json' 

{"id_task": "99808", "task": "text-to-image", "sd_model_checkpoint": "v1-5-pruned-emaonly.safetensors", "output_location": "s3://sdoneksdataplanestack-outputs3bucket9fe85b9f-pzlnjnu6xsl7/99808"}

In [ ]:
#查看结果

In [ ]:
s3uri = "s3://sagemaker-ap-northeast-1-544592066775/stable-diffusion-webui/generated/"

In [ ]:
#查看最近生成的随机的图片

In [ ]:
import boto3
import random
from PIL import Image
import requests
import io
from io import BytesIO

def get_latest_images(s3_uri, num_images):
    bucket_name, prefix = parse_s3_uri(s3_uri)

    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' in response:
        objects = response['Contents']
        objects.sort(key=lambda obj: obj['LastModified'], reverse=True)
        latest_objects = objects[:num_images]
        image_uris = [f"s3://{bucket_name}/{obj['Key']}" for obj in latest_objects]
        return image_uris

    return []

def _get_output_bucket_key(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    return bucket, key

def parse_s3_uri(s3_uri):
    parts = s3_uri.replace('s3://', '').split('/')
    bucket_name = parts[0]
    prefix = '/'.join(parts[1:])
    return bucket_name, prefix

# Example usage
s3_uri = s3uri
num_images = 5

latest_image_uris = get_latest_images(s3_uri, num_images)
if latest_image_uris:
    random_image_uri = random.choice(latest_image_uris)
    print("Randomly selected image URI:", random_image_uri)
    

    bucket, key = _get_output_bucket_key(random_image_uri)
    obj = s3_resource.Object(bucket, key)
    bytes = obj.get()['Body'].read()
    image = Image.open(io.BytesIO(bytes))
    image.show()
else:
    print("No images found.")
